In [151]:
##Assessment 2 MLPR
import numpy as np
import matplotlib.pyplot as plt
data = np.load('ct_data.npz')
X_train = data['X_train']; X_val = data['X_val']; X_test = data['X_test']
y_train = data['y_train']; y_val = data['y_val']; y_test = data['y_test']


In [152]:
########################## 1st question ####################################### 
print("Mean of y_train:",np.mean(y_train))
print("Standard error for y_train (for 5785 entries):",np.std(y_train[:5785], ddof=1)/np.sqrt(len(y_train[:5785])))
print("Mean of y_val (for 5785 entries):",np.mean(y_val))
print("Standard error for y_val (for 5785 entries):",np.std(y_val, ddof=1)/np.sqrt(len(y_val)))





########################## 1st question b part ###############################
rm_idx0=[]    

for i in range(len(X_train[1])):
    col=X_train[:,i]
    if all(col[0]==col):
        rm_idx0.append(i)
        
print("                                   ")
print("Removed column indices are(constants):",rm_idx0)

X_train = np.delete(X_train,rm_idx0,axis=1)
print("Modified shape of X_train:",X_train.shape)

X_val = np.delete(X_val,rm_idx0,axis=1)
print("Modified shape of X_val:",X_val.shape)

X_test = np.delete(X_test,rm_idx0,axis=1)
print("Modified shape of X_test:",X_test.shape)



indices= np.unique(X_train,return_index=True,axis=1)[1]
np.sort(indices)
rm_idx=list(set(range(indices[0],indices[-1]+1))-set(indices))

print("                                          ")
print("Removed column indices are:",rm_idx0+rm_idx)


X_train = np.delete(X_train,rm_idx,axis=1)
print("Modified shape of X_train:",X_train.shape)

X_val = np.delete(X_val,rm_idx,axis=1)
print("Modified shape of X_val:",X_val.shape)

X_test = np.delete(X_test,rm_idx,axis=1)
print("Modified shape of X_test:",X_test.shape)




Mean of y_train: -9.13868774539957e-15
Standard error for y_train (for 5785 entries): 0.011927303389170828
Mean of y_val (for 5785 entries): -0.2160085093241599
Standard error for y_val (for 5785 entries): 0.01290449880016868
                                   
Removed column indices are(constants): [59, 69, 179, 189, 351]
Modified shape of X_train: (40754, 379)
Modified shape of X_val: (5785, 379)
Modified shape of X_test: (6961, 379)
                                          
Removed column indices are: [59, 69, 179, 189, 351, 283]
Modified shape of X_train: (40754, 378)
Modified shape of X_val: (5785, 378)
Modified shape of X_test: (6961, 378)


In [136]:
k=len(X_train[1])
def fit_linreg(X, yy, alpha):
    yy = np.concatenate((yy, np.zeros(k)))
    z_k = np.sqrt(alpha) * np.eye(k)
    X=np.vstack((X,z_k))

    w_fit=np.linalg.lstsq(X, yy, rcond=None)[0]
    
    b = np.ones(len(X))
    f_grid= np.dot(X,w_fit)+b
    return f_grid, w_fit
    
    

In [137]:
alpha=30

ff, ww = fit_linreg(X_train, y_train, alpha)
yy = np.concatenate((y_train,np.ones(k)))

residuals = ff - yy
Error = np.sum(np.dot(residuals, residuals)) + alpha*np.dot(ww, ww)
print(Error)

46208.40565857615


In [138]:
## Support code
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import cho_factor, cho_solve

def params_unwrap(param_vec, shapes, sizes):
    """Helper routine for minimize_list"""
    args = []
    pos = 0
    for i in range(len(shapes)):
        sz = sizes[i]
        args.append(param_vec[pos:pos+sz].reshape(shapes[i]))
        pos += sz
    return args


def params_wrap(param_list):
    """Helper routine for minimize_list"""
    param_list = [np.array(x) for x in param_list]
    shapes = [x.shape for x in param_list]
    sizes = [x.size for x in param_list]
    param_vec = np.zeros(sum(sizes))
    pos = 0
    for param in param_list:
        sz = param.size
        param_vec[pos:pos+sz] = param.ravel()
        pos += sz
    unwrap = lambda pvec: params_unwrap(pvec, shapes, sizes)
    return param_vec, unwrap


def linreg_cost(params, X, yy, alpha):
    """Regularized least squares cost function and gradients

    Can be optimized with minimize_list -- see fit_linreg_gradopt for a
    demonstration.

    Inputs:
    params: tuple (ww, bb): weights ww (D,), bias bb scalar
         X: N,D design matrix of input features
        yy: N,  real-valued targets
     alpha: regularization constant

    Outputs: (E, [ww_bar, bb_bar]), cost and gradients
    """
    # Unpack parameters from list
    ww, bb = params

    # forward computation of error
    ff = np.dot(X, ww) + bb
    res = ff - yy
    E = np.dot(res, res) + alpha*np.dot(ww, ww)

    # reverse computation of gradients
    ff_bar = 2*res
    bb_bar = np.sum(ff_bar)
    ww_bar = np.dot(X.T, ff_bar) + 2*alpha*ww

    return E, [ww_bar, bb_bar]

def minimize_list(cost, init_list, args):
    """Optimize a list of arrays (wrapper of scipy.optimize.minimize)

    The input function "cost" should take a list of parameters,
    followed by any extra arguments:
        cost(init_list, *args)
    should return the cost of the initial condition, and a list in the same
    format as init_list giving gradients of the cost wrt the parameters.

    The options to the optimizer have been hard-coded. You may wish
    to change disp to True to get more diagnostics. You may want to
    decrease maxiter while debugging. Although please report all results
    in Q2-5 using maxiter=500.
    """
    opt = {'maxiter': 500, 'disp': False}
    init, unwrap = params_wrap(init_list)
    def wrap_cost(vec, *args):
        E, params_bar = cost(unwrap(vec), *args)
        vec_bar, _ = params_wrap(params_bar)
        return E, vec_bar
    res = minimize(wrap_cost, init, args, 'L-BFGS-B', jac=True, options=opt)
    return unwrap(res.x)


def fit_linreg_gradopt(X, yy, alpha):
    """
    fit a regularized linear regression model with gradient opt

         ww, bb = fit_linreg_gradopt(X, yy, alpha)

     Find weights and bias by using a gradient-based optimizer
     (minimize_list) to improve the regularized least squares cost:

       np.sum(((np.dot(X,ww) + bb) - yy)**2) + alpha*np.dot(ww,ww)

     Inputs:
             X N,D design matrix of input features
            yy N,  real-valued targets
         alpha     scalar regularization constant

     Outputs:
            ww D,  fitted weights
            bb     scalar fitted bias
    """
    D = X.shape[1]
    args = (X, yy, alpha)
    init = (np.zeros(D), np.array(0))
    ww, bb = minimize_list(linreg_cost, init, args)
    return ww, bb

In [139]:
ww1,bb=fit_linreg_gradopt(X_train, y_train, 30)

In [140]:
print(ww1)

[-6.14992475e-02 -1.05755951e-01  7.46755342e-02  2.78505333e-01
  2.59631223e-01  1.19646833e-01  1.11732455e-02  2.31129985e-01
 -3.15780079e-01 -7.66549193e-02 -4.08196100e-02  4.91191006e-02
 -2.16734631e-02 -1.87241954e-03 -8.19181615e-03  3.83573605e-02
  2.58478216e-02  5.21693455e-03 -8.43783363e-02  1.74539299e-01
  7.93132199e-02  2.82330041e-02 -4.11615371e-02 -5.52118908e-03
 -6.05811333e-02 -5.55979036e-02 -1.44667553e-02 -4.12151439e-01
  3.47778732e-01  2.61275433e-01 -1.31062922e-02  2.13081181e-03
 -8.53377987e-03 -7.39516602e-03  6.49916325e-02 -1.76305165e-01
 -1.79920114e-01  2.19650510e-01 -8.91029953e-02 -3.01615683e-02
 -1.80493371e-01  1.86934668e-02 -1.75219748e-01  5.29462008e-02
 -1.39803039e-01 -2.19871595e-02 -1.93785971e-02  1.86829281e-02
 -3.68071194e-01  4.45258261e-02 -9.20498763e-02 -2.46670359e-03
 -7.85443668e-02 -5.90617139e-02 -3.57144806e-02 -7.94930520e-02
  5.05835081e-02 -1.29454815e-01  7.95511280e-02 -7.81503651e-02
  6.29905095e-02 -1.38048

In [142]:
print(ww-ww1)

[-3.96140980e-04 -3.98461337e-04  4.97011707e-04 -5.31069139e-05
  6.87643221e-04  7.66041208e-05  1.15092094e-04  5.15532974e-04
  4.17381810e-04 -3.14496490e-03  3.44960491e-04  7.40750387e-05
  6.35594518e-04 -1.25009716e-04 -3.87360902e-04  2.34760120e-04
  6.79772363e-04 -1.16853484e-03  4.15373580e-04  5.41907401e-04
 -3.84290502e-04  1.16036821e-04  2.70365716e-04 -1.61095797e-04
  1.26065034e-04 -2.23524249e-04  1.19114677e-03  2.19151043e-03
 -3.03768294e-03 -4.82009651e-03 -8.69431909e-04 -2.85183312e-04
 -4.15392855e-04  1.31239198e-04 -6.44274973e-05 -1.63338053e-04
  1.64629068e-04 -2.14459992e-03 -1.41032418e-03 -3.04967691e-03
  8.18348846e-04  3.14984932e-04  4.88373389e-04 -2.51259524e-04
  1.07518844e-03  7.88524425e-05 -1.62943122e-03  1.45999907e-04
 -1.73323951e-02 -7.03491056e-03  3.82436630e-04  2.14212036e-04
  1.95932721e-04  4.37009169e-06  2.97226138e-04  4.87155525e-07
  1.17716303e-03  2.92252088e-03 -7.39599272e-03 -3.84382605e-04
  9.84231034e-05 -2.44532

In [135]:
print(bb)

0.07991036323816879
